In [ ]:
#class for getting & parsing RSS feeds
import feedparser as fp
import pandas as pd
import html
from nltk.tokenize import TreebankWordTokenizer,WhitespaceTokenizer
import string
from nltk.corpus import stopwords
import re
import time
from guess_language import guess_language
import time
import numpy as np

WORD = re.compile(r'\w+')


#These functions clean the data in various ways
class Cleaner():
    
    def __init__(self):
        self.wp = WhitespaceTokenizer()
        self.reclean = re.compile('<.*?>')
        
    @classmethod
    def remove_html_tags(cls,text):
        import re
        return re.sub(self.reclean, '', text)   
    
    @classmethod
    def replace_dash(cls,text):
        return text.replace('-','')

    #This removes non-alphabetical characters and makes everything lower case
    @classmethod
    def clean(cls,text):
        return ''.join(c for c in cls.remove_html_tags(cls.replace_dash(html.unescape(text.lower()))) 
                       if c in string.ascii_lowercase+' ')
    #this tokenizes intelligently

    def tokenize(self,text):
        #return TreebankWordTokenizer().tokenize(text)
        return self.wp.tokenize(text)
    #     words = WORD.findall(text)
    #     return words
    #this removes stopword tokens from a list of tokens
    def remove_stop_words(self,tokens):
        return [word for word in tokens if word not in stopwords.words('english')]
    #this will clean & tokenize a list of documents.

    def preprocess_documents(self,summaries):
        return [self.remove_stop_words(self.tokenize(self.clean(s))) for s in summaries]
          

In [6]:
#Scan output folder for pickle files w/ identifier and clean them. 
import time
from multiprocessing import Pool
import pickle

floc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries2/'
foutloc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries3/'
identifier = '_1529184593.957413'
file_prefix = "feeds_"
file_suffix = ".pkl"

#preprocessing wrapper function for multiprocessing
def pp(summaries):
    return (summaries[0],cleaner.preprocess_documents(summaries[1]))

cleaner = Cleaner()

nfeed_files = 276

if __name__ == '__main__':
    p = Pool(8)
    for i in range(275,nfeed_files):
        cleaned_feeds = []
        #load in file
        fname = file_prefix + str(i) + identifier + file_suffix
        with open(floc+fname,'rb') as fid:
            thisfeeds = pickle.load(fid)

        start_time = time.time()
        cleaned_feeds = p.map(pp,thisfeeds)

        with open(foutloc+'preprocessed_summaries'+str(i)+identifier+file_suffix,'wb') as fid:
            pickle.dump(cleaned_feeds,fid)

        stop_time = time.time()
        duration = stop_time - start_time
        print('Parsed file %s, duration = %.2f' % (fname,duration))
        i += 1



Parsed file feeds_275_1529184593.957413.pkl, duration = 17.37


In [8]:
#Combine groups of 50 files, and combine preprocessed descriptions into strings. 
#Reduces the number of files and saves space for RAM efficiency. 
import time
import pickle

floc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries3/'
foutloc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries3_2/'
associator = '_1529184593.957413'
ext = '.pkl'

file_chunk_size = 50
nfile = 276
fctr = 0
for fr in range(0,nfile,file_chunk_size):
    preprocessed_summaries = []
    for i in range(fr,fr+file_chunk_size):

        start_time = time.time()

        fname = 'preprocessed_summaries' + str(i) + associator + ext
        with open(floc+fname,'rb') as fid:
            thisdata = pickle.load(fid)

        for t in thisdata:
            for j in range(0,len(t[1])):
                t[1][j] = ' '.join(t[1][j])
            preprocessed_summaries.append(t)

        stop_time = time.time()
        duration = stop_time - start_time
        print('loaded file %d, duration = %.2f' % (i,duration))
    print('saving chunk %d...' % fr)
    with open(foutloc + 'preprocessed_summaries' + str(fctr) + associator + ext,'wb') as fid:
        pickle.dump(preprocessed_summaries,fid)
        fctr += 1

loaded file 0, duration = 0.12
loaded file 1, duration = 0.15
loaded file 2, duration = 0.13
loaded file 3, duration = 0.12
loaded file 4, duration = 0.20
loaded file 5, duration = 0.13
loaded file 6, duration = 0.13
loaded file 7, duration = 0.11
loaded file 8, duration = 0.14
loaded file 9, duration = 0.14
loaded file 10, duration = 0.15
loaded file 11, duration = 0.13
loaded file 12, duration = 0.13
loaded file 13, duration = 0.11
loaded file 14, duration = 0.15
loaded file 15, duration = 0.11
loaded file 16, duration = 0.16
loaded file 17, duration = 0.15
loaded file 18, duration = 0.13
loaded file 19, duration = 0.15
loaded file 20, duration = 0.15
loaded file 21, duration = 0.12
loaded file 22, duration = 0.17
loaded file 23, duration = 0.14
loaded file 24, duration = 0.15
loaded file 25, duration = 0.17
loaded file 26, duration = 0.27
loaded file 27, duration = 0.18
loaded file 28, duration = 0.21
loaded file 29, duration = 0.23
loaded file 30, duration = 0.14
loaded file 31, du

loaded file 250, duration = 0.39
loaded file 251, duration = 0.49
loaded file 252, duration = 0.50
loaded file 253, duration = 0.35
loaded file 254, duration = 0.34
loaded file 255, duration = 0.40
loaded file 256, duration = 0.14
loaded file 257, duration = 0.13
loaded file 258, duration = 0.15
loaded file 259, duration = 0.15
loaded file 260, duration = 0.12
loaded file 261, duration = 0.14
loaded file 262, duration = 0.09
loaded file 263, duration = 0.16
loaded file 264, duration = 0.13
loaded file 265, duration = 0.13
loaded file 266, duration = 0.20
loaded file 267, duration = 0.13
loaded file 268, duration = 0.16
loaded file 269, duration = 0.14
loaded file 270, duration = 0.12
loaded file 271, duration = 0.13
loaded file 272, duration = 0.11
loaded file 273, duration = 0.15
loaded file 274, duration = 0.21
loaded file 275, duration = 0.06


FileNotFoundError: [Errno 2] No such file or directory: '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries3/preprocessed_summaries276_1529184593.957413.pkl'

loaded file 276, duration = 54.48
saving chunk 250...


Fit a TF-IDF model for smarter weighting. 

In [1]:
#load in for tf-idf
import time
import pickle

floc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries3_2/'
associator = '_1529184593.957413'
ext = '.pkl'

file_chunk_size = 50
nfile = 6
preprocessed_summaries = []
for i in range(0,nfile):

    start_time = time.time()

    fname = 'preprocessed_summaries' + str(i) + associator + ext
    with open(floc+fname,'rb') as fid:
        thisdata = pickle.load(fid)

    for t in thisdata:
        preprocessed_summaries.append(t)

    stop_time = time.time()
    duration = stop_time - start_time
    print('loaded file %d, duration = %.2f' % (i,duration))
    
print('Done! (len = %d)' % len(preprocessed_summaries))

summary_list = []
for d in preprocessed_summaries:
    summary_list += d[1]
    

loaded file 0, duration = 0.64
loaded file 1, duration = 0.45
loaded file 2, duration = 0.51
loaded file 3, duration = 0.30
loaded file 4, duration = 0.33
loaded file 5, duration = 0.25
Done! (len = 79716)


In [5]:
import spacy

nlp = spacy.load('en_core_web_sm')

preprocessed_summaries[0][1][0]

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [2]:
# import gensim.downloader as api
# from gensim.models import TfidfModel
# from gensim.corpora import Dictionary
# from functools import reduce

# #dataset = reduce(lambda x,y: x+y[1],preprocessed_summaries)


# dct = Dictionary()
# chunk_size = 500000
# for i in range(0,len(summary_list),chunk_size):
#     print(i)
#     dct.add_documents([s.split() for s in summary_list[i:(i+chunk_size)]])

# floc = '/home/bmassi/Dropbox/professional/Insight/data/'
# fname = 'tfidfdict.pkl'
# with open(floc+fname,'wb') as fid:
#     pickle.dump(dct,fid)

In [2]:
# floc = '/home/bmassi/Dropbox/professional/Insight/data/'
# fname = 'tfidfdict.pkl'
# with open(floc+fname,'rb') as fid:
#     dct = pickle.load(fid)

In [3]:
# corpus = []
# chunk_size = 500000
# for i in range(0,len(summary_list),chunk_size):
#     print(i)
#     corpus += [dct.doc2bow(line) for line in [s.split() for s in summary_list[i:(i+chunk_size)]]]  # convert dataset to BoW format


In [ ]:
# model = TfidfModel(corpus)  # fit model
# vector = model[corpus[0]]  # apply model

In [1]:
#load in word2vec, and define functions for parsing tokenized episode summaries.
import gensim
import numpy as np

floc = '/home/bmassi/Dropbox/professional/Insight/data/'
fname = 'GoogleNews-vectors-negative300.bin'

word2vec = gensim.models.KeyedVectors.load_word2vec_format(floc+fname, binary=True)

print(word2vec['apples'])

[-0.14355469 -0.00723267 -0.00196838  0.07958984 -0.08398438  0.03540039
  0.18847656 -0.02246094 -0.140625    0.1796875   0.09716797 -0.10498047
 -0.08349609  0.14941406 -0.29492188  0.06835938 -0.07861328  0.00145721
 -0.1484375  -0.12060547 -0.20703125  0.24316406  0.12158203  0.17285156
  0.12890625 -0.09521484 -0.30273438  0.22851562 -0.25390625  0.02478027
 -0.18652344 -0.04956055  0.06005859  0.17871094  0.10742188  0.09570312
 -0.00361633 -0.328125    0.18847656  0.24609375  0.16503906 -0.171875
  0.00701904 -0.42578125  0.00402832 -0.265625   -0.00701904  0.05151367
 -0.17578125  0.0402832   0.00125122  0.04516602  0.02575684 -0.05541992
 -0.1640625  -0.01403809 -0.11816406 -0.05932617 -0.06201172 -0.375
  0.06396484  0.18066406 -0.08642578  0.09521484  0.02661133 -0.16015625
 -0.08300781 -0.25976562 -0.05639648 -0.02709961  0.14941406  0.02209473
 -0.11474609 -0.28320312 -0.14160156 -0.04296875 -0.23925781 -0.04272461
  0.17089844  0.02844238 -0.25390625  0.140625   -0.095703

In [2]:
#define functions for vectorizing words

#takes list of words, safely computes average word2vec. 
# def ep2vec(episode):
#     n = 0
#     a = np.array([300,])
#     for w in episode:
#         try:
#             v = word2vec[w]
#             if((np.isnan(v).any() + np.isinf(v).any()) == 0):
#                 a = np.add(a,v)
#                 n += 1
#         except KeyError as e:
#             pass
#     if(n==0):
#         return np.full([300,], np.nan)
#     avg = a/n
#     return avg


def ep2vec(episode):
    n = 0
    a = []
    for w in episode:
        try:
            v = word2vec[w]
            if((np.isnan(v).any() + np.isinf(v).any()) == 0):
                a.append(v)
                n += 1
        except KeyError as e:
            pass
    if(n==0):
        return np.full([300,], np.nan)
    #print(np.array(a).shape)
    avg = np.mean(np.array(a),axis=0)
    return avg
    

# #takes lists of episodes, vectorizes them, then averages the vectors
# def pod2vec(episodes):
#     v = []
#     for e in episodes:
#         thisv = ep2vec(e)
#         if(not np.isnan(thisv).any()):
#             v.append(thisv)
#     return np.mean(v)

#takes lists of episodes, vectorizes them, then averages the vectors
def pod2vec(episodes):
    return np.nanmean(np.asarray([ep2vec(e) for e in episodes]),axis=0)

In [9]:
#load in preprocessed text summaries and word2vec them
import pickle
import time

floc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries3/'#100
#floc = '/home/bmassi/Dropbox/professional/Insight/data/preprocessed_summaries2/'#1k
fctr = 0
associator = '1529184593.957413'#'1528891839'#100,1k
nfile = 276
vectorized_eps = []
for i in range(0,nfile):
    start_time = time.time()
    #fname = 'preprocessed_summaries' + str(fctr) +'_'+ associator + '.pkl'#1k
    fname = 'preprocessed_summaries' + str(fctr) +'_'+ associator + '.pkl'#100
    with open(floc+fname,'rb') as fid:
         pps = pickle.load(fid)
    for p in pps:
        vectorized_eps.append((p[0],pod2vec(p[1]),[ep2vec(e) for e in p[1]]))
    fctr += 1
    stop_time = time.time()
    duration = stop_time - start_time
    print('Done with file ' + str(i) + ' (duration=' + str(duration) + ')')
print('done!')


Done with file 0 (duration=14.686335802078247)
Done with file 1 (duration=18.17879366874695)
Done with file 2 (duration=16.047038555145264)
Done with file 3 (duration=15.189306259155273)


/home/bmassi/anaconda3/envs/insight/lib/python3.6/site-packages/ipykernel/__main__.py:50: RuntimeWarning: Mean of empty slice


Done with file 4 (duration=22.405528783798218)
Done with file 5 (duration=17.550069332122803)
Done with file 6 (duration=16.985368013381958)
Done with file 7 (duration=15.59252119064331)
Done with file 8 (duration=19.830849647521973)
Done with file 9 (duration=18.209243535995483)
Done with file 10 (duration=16.46628975868225)
Done with file 11 (duration=16.739453077316284)
Done with file 12 (duration=15.851783037185669)
Done with file 13 (duration=13.98745059967041)
Done with file 14 (duration=18.417743682861328)
Done with file 15 (duration=15.651981353759766)
Done with file 16 (duration=17.00659966468811)
Done with file 17 (duration=19.566552877426147)
Done with file 18 (duration=15.849315881729126)
Done with file 19 (duration=21.136651277542114)
Done with file 20 (duration=18.246132373809814)
Done with file 21 (duration=16.17673349380493)
Done with file 22 (duration=20.66201663017273)
Done with file 23 (duration=17.893476247787476)
Done with file 24 (duration=17.966702222824097)
Done

Done with file 175 (duration=6.234348773956299)
Done with file 176 (duration=10.947169780731201)
Done with file 177 (duration=8.238808870315552)
Done with file 178 (duration=5.882059574127197)
Done with file 179 (duration=8.772454261779785)
Done with file 180 (duration=6.338480472564697)
Done with file 181 (duration=6.7168943881988525)
Done with file 182 (duration=10.484297752380371)
Done with file 183 (duration=7.631116628646851)
Done with file 184 (duration=6.552612543106079)
Done with file 185 (duration=7.387449741363525)
Done with file 186 (duration=7.04577112197876)
Done with file 187 (duration=6.236070156097412)
Done with file 188 (duration=5.670293092727661)
Done with file 189 (duration=12.752849817276001)
Done with file 190 (duration=9.05562138557434)
Done with file 191 (duration=6.661819934844971)
Done with file 192 (duration=9.38100004196167)
Done with file 193 (duration=7.3947649002075195)
Done with file 194 (duration=4.2695441246032715)
Done with file 195 (duration=10.31354

In [10]:
#save vector representations

floc = '/home/bmassi/Dropbox/professional/Insight/data/vectorized_summaries3/'

chunk_size = 1000
fctr = 0
associator = str(int(np.floor(time.time())))
for i in range(0,len(vectorized_eps),chunk_size):
    print('File '+str(fctr))
    fname = 'vectorized_summaries' + str(fctr) +'_'+ associator + '.pkl'
    with open(floc+fname,'wb') as fid:
         pickle.dump(vectorized_eps[i:(i+chunk_size)],fid)
    print(i)
    fctr += 1

File 0
0
File 1
1000
File 2
2000
File 3
3000
File 4
4000
File 5
5000
File 6
6000
File 7
7000
File 8
8000
File 9
9000
File 10
10000
File 11
11000
File 12
12000
File 13
13000
File 14
14000
File 15
15000
File 16
16000
File 17
17000
File 18
18000
File 19
19000
File 20
20000
File 21
21000
File 22
22000
File 23
23000
File 24
24000
File 25
25000
File 26
26000
File 27
27000
File 28
28000
File 29
29000
File 30
30000
File 31
31000
File 32
32000
File 33
33000
File 34
34000
File 35
35000
File 36
36000
File 37
37000
File 38
38000
File 39
39000
File 40
40000
File 41
41000
File 42
42000
File 43
43000
File 44
44000
File 45
45000
File 46
46000
File 47
47000
File 48
48000
File 49
49000
File 50
50000
File 51
51000
File 52
52000
File 53
53000
File 54
54000
File 55
55000
File 56
56000
File 57
57000
File 58
58000
File 59
59000
File 60
60000
File 61
61000
File 62
62000
File 63
63000
File 64
64000
File 65
65000
File 66
66000
File 67
67000
File 68
68000
File 69
69000
File 70
70000
File 71
71000
File 72
72000
F

In [7]:
#Load in vectorized summaries
import pickle
import time

floc = '/home/bmassi/Dropbox/professional/Insight/data/vectorized_summaries3/'#100

fctr = 0
associator = '1529332905'#'1528891839'#100,1k
nfile = 80
vectorized_eps = []
for i in range(0,nfile):
    start_time = time.time()
    #fname = 'preprocessed_summaries' + str(fctr) +'_'+ associator + '.pkl'#1k
    fname = 'vectorized_summaries' + str(fctr) +'_'+ associator + '.pkl'#100
    with open(floc+fname,'rb') as fid:
         pps = pickle.load(fid)
    for p in pps:
        vectorized_eps.append(p[0:3])
    fctr += 1
    stop_time = time.time()
    duration = stop_time - start_time
    print('Done with file ' + str(i) + ' (duration=' + str(duration) + ')')
print('done!')
len(vectorized_eps)

Done with file 0 (duration=0.13778448104858398)
Done with file 1 (duration=0.12509822845458984)
Done with file 2 (duration=0.1330428123474121)
Done with file 3 (duration=0.12988018989562988)
Done with file 4 (duration=0.12023305892944336)
Done with file 5 (duration=0.1210031509399414)
Done with file 6 (duration=0.1453394889831543)
Done with file 7 (duration=0.13929271697998047)
Done with file 8 (duration=0.12029099464416504)
Done with file 9 (duration=0.12156915664672852)
Done with file 10 (duration=0.1341841220855713)
Done with file 11 (duration=0.11548113822937012)
Done with file 12 (duration=0.1252601146697998)
Done with file 13 (duration=0.12752938270568848)
Done with file 14 (duration=0.13009285926818848)
Done with file 15 (duration=0.14284396171569824)
Done with file 16 (duration=0.11864519119262695)
Done with file 17 (duration=0.11605381965637207)
Done with file 18 (duration=0.1242210865020752)
Done with file 19 (duration=0.11220622062683105)
Done with file 20 (duration=0.129658

In [1]:
import pickle

floc = '/home/bmassi/Dropbox/professional/Insight/data/'
fname = 'podcast_df_subset_BIGDATA_1529347011.pkl'

with open(floc+fname,'rb') as fid:
    podcast_df = pickle.load(fid)

numpy.ndarray